<a href="https://colab.research.google.com/github/matlogica/AADC-Python/blob/main/QuantLib/07-LiveRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AADC Live risk server approach
We start with generating a random portfolio of 1000 IR swaps with random start dates, notionals and maturities.
We then record an AADC kernel for a single portfolio price, marking the IR zero rates as inputs.
We then simulate random "market updates" and demonstrate fast repricing of the portfolio along with bucketed AAD deltas.
Kernel execution is so fast (within tens of ms), that it can be used as a source of a ticking "live risk" for the portfolio

In [17]:
import aadc
import aadc.quantlib as ql
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
import random
import time

Random portfolio generation:

In [18]:
def createRandomSwaps(num_trades, todays_date, calendar):
    random.seed(42)
    portfolio = []

    for _ in range(num_trades):
        discounting_term_structure = ql.RelinkableYieldTermStructureHandle()
        forecasting_term_structure = ql.RelinkableYieldTermStructureHandle()

        nominal = 1000000.0 * (100.0 + random.randint(0, 399)) * (-1.0 + 2 * random.randint(0, 1))

        # Fixed leg
        fixed_leg_frequency = ql.Bimonthly
        fixed_leg_convention = ql.Unadjusted
        floating_leg_convention = ql.ModifiedFollowing
        fixed_leg_day_counter = ql.Thirty360(ql.Thirty360.European)
        fixed_rate = 0.03
        floating_leg_day_counter = ql.Actual360()
        start_date = todays_date + ql.Period(10 + random.randint(0, 359), ql.Days)

        # Floating leg
        floating_leg_frequency = ql.Semiannual
        euribor_index = ql.Euribor6M(forecasting_term_structure)
        spread = 0.0

        length_in_years = 2 + random.randint(0, 25)

        swap_type = ql.VanillaSwap.Payer

        maturity_date = start_date + ql.Period(length_in_years, ql.Years)
        fixed_schedule = ql.Schedule(start_date, maturity_date, ql.Period(fixed_leg_frequency), calendar,
                                     fixed_leg_convention, fixed_leg_convention, ql.DateGeneration.Forward, False)
        floating_schedule = ql.Schedule(start_date, maturity_date, ql.Period(floating_leg_frequency), calendar,
                                        floating_leg_convention, floating_leg_convention, ql.DateGeneration.Forward, False)

        random_vanilla_swap = ql.VanillaSwap(
            swap_type, nominal, fixed_schedule, fixed_rate, fixed_leg_day_counter,
            floating_schedule, euribor_index, spread, floating_leg_day_counter
        )

        swap_engine = ql.DiscountingSwapEngine(discounting_term_structure)
        random_vanilla_swap.setPricingEngine(swap_engine)

        portfolio.append((random_vanilla_swap, discounting_term_structure, forecasting_term_structure))

    return portfolio

Pricing using original Quantlib code

In [19]:
def price_portfolio(portfolio, dates, zero_rates):
    total_npv = aadc.idouble(0.0)

    log_linear_curve = ql.ZeroCurve(dates, zero_rates.tolist(), ql.Actual360(), ql.TARGET())
    log_linear_curve.enableExtrapolation()

    for (swap, discounting_term_structure, forecasting_term_structure) in portfolio:
        discounting_term_structure.linkTo(log_linear_curve)
        forecasting_term_structure.linkTo(log_linear_curve)
        total_npv += swap.NPV()

    return total_npv

Recording the AADC kernel

In [20]:


def record_kernel(portfolio, dates):
    kernel = aadc.Kernel()
    kernel.start_recording()

    zero_rates = aadc.array(np.zeros(len(dates)))
    zero_args = zero_rates.mark_as_input()

    total_npv = price_portfolio(portfolio, dates, zero_rates)

    res = total_npv.mark_as_output()
    kernel.stop_recording()

    return (kernel, { res: zero_args })

Set up everything and run the recording. This could be done at the beginning of a trading day in a live risk server setting

In [21]:

num_trades = 1000
todays_date = ql.Date(12, ql.June, 2024)
calendar = ql.TARGET()

portfolio = createRandomSwaps(num_trades, todays_date, calendar)
dates = [todays_date] + [todays_date + ql.Period(i, ql.Years) for i in range(1, 30)]

mark_time = time.time()
(kernel, request) = record_kernel(portfolio, dates)

print("Recording time: ", time.time() - mark_time)

kernel.print_passive_extract_locations()

You are using evaluation version of AADC. Expire date is 20240901
Recording time:  4.090595722198486
Number active to passive conversions: 0 while recording Python


Time one portfolio pricing with original Quantlib code

In [22]:
%%time
price_portfolio(portfolio, dates, np.zeros(len(dates)))

CPU times: user 70.1 ms, sys: 33 ms, total: 103 ms
Wall time: 103 ms


-4826369999.999888

Execute the recorded kernel for a given market update. The random "market update" is simulated by providing different seeds to the random numbers generator and subsequently using the generator to produce zero rates for the IR curves.

In [23]:
import time

def calc_risks(seed):
    random.seed(seed)
    zero_rates = np.zeros(len(dates))
    for i in range(len(dates)):
        zero_rates[i] = 0.0025 + 0.005 * 0.02 * random.randint(0, 99)

    args = list(request.values())[0]

    time_mark = time.time()

    r = aadc.evaluate(kernel, request, { a: [x] for a, x in zip(args, zero_rates) }, aadc.ThreadPool(1))

    print("Calculation time: ", time.time() - time_mark)

    return (zero_rates, r)

calc_risks(42)

Calculation time:  0.05907607078552246


(array([0.0106, 0.0039, 0.0028, 0.0119, 0.006 , 0.0056, 0.0053, 0.0042,
        0.0119, 0.0038, 0.0111, 0.0119, 0.0094, 0.0036, 0.01  , 0.0079,
        0.0029, 0.0028, 0.0036, 0.0052, 0.0054, 0.0089, 0.0102, 0.0028,
        0.0096, 0.005 , 0.0116, 0.0108, 0.0114, 0.0094]),
 ({Res(593280): array([-3.06171282e+09])},
  {Res(593280): {Arg(7): array([-2.71154029e+08]),
    Arg(8): array([-2.33599052e+09]),
    Arg(9): array([95473624.52143878]),
    Arg(10): array([-2.67159216e+09]),
    Arg(11): array([-1.13067702e+10]),
    Arg(12): array([-7.7708617e+09]),
    Arg(13): array([-1.68327627e+09]),
    Arg(14): array([-5.02848577e+08]),
    Arg(15): array([1.40243253e+09]),
    Arg(16): array([1.88911676e+09]),
    Arg(17): array([7.14903345e+09]),
    Arg(18): array([3.85957032e+09]),
    Arg(19): array([-1.30158466e+10]),
    Arg(20): array([-3.09489203e+09]),
    Arg(21): array([-2.93155413e+10]),
    Arg(22): array([2.19427717e+10]),
    Arg(23): array([1.77834842e+10]),
    Arg(24): ar

Plot the results interactively

In [24]:
def plot_risks(seed):
    (zero_rates, r) = calc_risks(seed)

    risks = [item for subdict in r[1].values() for sublist in subdict.values() for item in sublist]
    assert(len(risks) == len(dates))

    plot_years = range(0, 30)
    assert(len(plot_years) == len(dates))

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    plt.plot(plot_years, zero_rates, marker='o', linestyle='-', color='b')
    plt.xlabel('Year')
    plt.ylabel('Zero Rate')
    plt.title('Zero Rate Curve')
    plt.grid(True)
    plt.ylim(0, max(zero_rates) * 1.5)

    # Plotting the riks
    plt.subplot(1, 2, 2)
    plt.plot(plot_years, risks, marker='o', linestyle='-', color='r')
    plt.xlabel('Year')
    plt.ylabel('Zero risk')
    plt.title('Zero risks')
    plt.grid(True)
    plt.ylim(min(risks) * 1.5, max(risks) * 1.5)

    plt.tight_layout()
    plt.show()



# Create interactive sliders for level and slope
seed_slider = widgets.FloatSlider(
    value=42,
    min=1,
    max=100,
    step=1,
    description='Seed:',
    continuous_update=False
)

# Use interactive function to update the plot
interactive_plot = interactive(plot_risks, seed=seed_slider)
output = interactive_plot.children[-1]
output.layout.height = '650px'
interactive_plot


interactive(children=(FloatSlider(value=42.0, continuous_update=False, description='Seed:', min=1.0, step=1.0)…